# Compatison of green portfolios vs non-green based on the constituents of SP500 stokcs

# General description of the expiriment

## For this expiriment i constructed portfolio using constituents of SP500 stocks. I rely on classical mean-variance portfolio optimization theory usnig historical mean returns of the stocks based on their daily adjusted closing prices and applying Ledoit Wolf shrinkage to covariance matrix. The stock prices ranges from 2014-01-01 till 2016-09-01.



## I am also using The Robeco Total Sustainability Rank Envoronmental score to construct green portfolio: in addition to traditional constraints the weighted sum of environmental score of stocks included in the portfolio should be more or equal some seted treshold. The score ranges from 0 to 100: the higher the score - more environmetally friendly stock/portfolio is. 



## The environmetal score used for calculation is dated as September 2016.



## This score rates stocks based on company responses to questionnaires regarding their practices across the  Environmental component of ESG dimension. RobecoSAM crafts and sends the questionnaires to companies in cooperation with Standard and Poor’s (S&P). RobecoSAM, part of the Robeco group, is “an investment specialist focused exclusively on sustainability investing” (Bloomberg Financial Terminal). Like the Bloomberg and Sustainalytics rankings, the Robeco scores make adjustments by industry. “The types and weights of individual questions and criteria are adjusted for each industry-specific questionnaire to reflect the materiality of specific sustainability themes within each industry” (Bloomberg Financial Terminal).

## The job was done using pyportfoliopt package which offers a lot of useful instruments for portfolio optimization

# Importing libraries

In [9]:
#pip install pyportfolioopt

In [1]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [4]:
import datetime
import pandas_datareader as web

ModuleNotFoundError: No module named 'pandas_datareader'

In [2]:
from pypfopt import EfficientSemivariance

In [6]:
import yfinance as yffrom 
from pypfopt.expected_returns import mean_historical_return
from pypfopt.expected_returns import ema_historical_return
from pypfopt.risk_models import CovarianceShrinkage

ModuleNotFoundError: No module named 'yfinance'

# Getting and preparing the data

In [3]:
#SP500

In [5]:
sp500url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [6]:
data_table=pd.read_html(sp500url)

In [7]:
TABLE=data_table[0]

In [8]:
TABLE

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [9]:
# Getting data for ESG scores

In [10]:
data_esg=pd.read_excel('/Users/vladimirant/Downloads/ESG_Score.xlsx')

In [11]:
data_esg.describe()

,E_score
count,419.000000
mean,44.040573
std,26.275332
min,0.000000
25%,22.000000
50%,42.000000
75%,65.000000
max,100.000000


In [12]:
# Merging data: TAble + ESG scores. We left with only with tickers that have ESG scores.
# Tichers without ESG scores are ecluded. 375 stocks left in our portfolio universe.

In [13]:
merged_data=pd.merge(TABLE, data_esg, on='Symbol')

In [14]:
Tickers=merged_data['Symbol'].tolist()

In [15]:
Tickers

['MMM',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AAPL',
 'AAPL',
 'AMAT',
 'APTV',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BAC',
 'BAX',
 'BDX',
 'BBY',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'CHRW',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBRE',
 'CE',
 'CNC',
 'CNP',
 'CF',
 'SCHW',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CINF',
 'CTAS',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'GLW',
 'COST',
 'CCI',
 'CSX',
 'CMI',
 'CVS',
 'DHI',
 'DHR',
 'DRI',
 'DVA',
 'DE',
 'DAL',
 'XRAY',
 'DVN',
 'DLR',
 'DFS',
 'DISH',
 'DG',
 'DLTR',
 'D',
 'DOV',
 'DTE',
 'DUK',
 'EMN',
 'ETN',
 'EBAY',
 'ECL',
 'EI

In [16]:
len(Tickers)

373

In [18]:
# Web scrapping stocks data

In [20]:
#Setting up start and end date

start='2014-01-01'
end='2016-09-01'


In [21]:
scrapping=yf.download(Tickers, start,end)

[*********************100%***********************]  372 of 372 completed


In [22]:
snp_prices=scrapping['Adj Close']

In [26]:
df_test=snp_prices.copy()


In [23]:
df=snp_prices.copy()

In [24]:
snp_prices

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,WMT,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00-05:00,37.141602,23.907925,101.185295,17.364958,35.240074,60.331604,31.847374,68.658340,59.290001,40.111164,...,64.243126,21.375246,168.754120,20.609205,66.009567,44.643024,30.301588,45.261662,83.170258,30.264683
2014-01-03 00:00:00-05:00,37.610737,25.020359,104.080521,16.983528,35.457016,60.374783,32.188923,68.886841,59.160000,40.379757,...,64.031425,21.423101,167.100159,20.601713,65.850731,44.857117,30.576565,45.544952,83.530907,29.974749
2014-01-06 00:00:00-05:00,37.425732,25.482302,103.084709,17.076141,34.162128,60.158966,32.613792,68.159050,58.119999,40.151871,...,63.673214,21.204353,166.980804,20.489336,65.950020,44.587181,30.523352,45.508793,84.071922,29.909281
2014-01-07 00:00:00-05:00,37.960964,25.369171,104.357132,16.954023,34.229931,60.815025,32.363865,68.988403,58.970001,40.363487,...,63.868618,21.265879,171.797668,20.841440,66.883064,45.275997,30.612051,46.147713,85.749023,30.021509
2014-01-08 00:00:00-05:00,38.582081,26.047943,103.545738,17.061382,34.141781,61.410648,32.655434,69.521538,58.900002,40.461178,...,63.363884,21.067640,175.020355,20.759035,66.664696,45.359779,30.594313,46.129631,87.849930,29.684818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-25 00:00:00-04:00,44.787663,35.394398,148.295074,24.966097,48.146824,76.622826,37.957409,103.006233,101.690002,55.169922,...,62.414619,23.647236,83.173439,34.726860,63.174366,57.340115,46.587749,57.569401,118.034012,48.274261
2016-08-26 00:00:00-04:00,44.844532,34.922085,145.193604,24.819874,48.064987,76.837875,38.081444,103.149490,102.059998,55.381023,...,62.344524,23.317343,82.566933,33.995861,63.037113,57.887299,46.670635,57.306973,119.277328,48.551640
2016-08-29 00:00:00-04:00,45.128841,34.864250,145.971313,24.792028,47.990601,77.339684,38.320667,102.836105,102.300003,55.319454,...,62.572365,23.722208,83.366417,34.250484,63.448853,58.368439,47.100243,58.094273,120.272003,48.551640


# Calculating mean returns and covariance matrix

In [34]:


mu = mean_historical_return(df)
mu_e= ema_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()


# Portfolio Optimization under various scenarious

In [35]:
# Target volatility=0.1

ef = EfficientFrontier(mu, S, weight_bounds=(0, 1)) 
weights = ef.efficient_risk(target_volatility=0.1)

In [36]:
PO=ef.clean_weights()

In [37]:
#tickers for portfolio constituents

res = [d for d in PO if PO[d] > 0]
res

['ABC',
 'AWK',
 'CLX',
 'CTAS',
 'DLR',
 'EA',
 'ED',
 'EW',
 'HPE',
 'HUM',
 'ISRG',
 'KHC',
 'LMT',
 'MKTX',
 'MO',
 'NEM',
 'NVDA',
 'O',
 'ORLY',
 'RSG',
 'SO',
 'STZ',
 'SYF',
 'SYY',
 'T',
 'TDG',
 'TSN',
 'WM']

In [38]:
#percentage share of portfolio constituents

for i in res:
    print(PO.get(i)*100)

2.871
6.279999999999999
4.8759999999999994
1.006
3.549
2.445
3.8600000000000003
3.2009999999999996
12.034
0.06899999999999999
1.49
11.996
6.069
0.22599999999999998
2.707
2.672
2.0549999999999997
5.537
0.35500000000000004
0.943
4.5120000000000005
1.95
0.457
5.563
2.619
1.8319999999999999
3.066
5.761


In [483]:
ef.portfolio_performance(verbose=True)

Expected annual return: 28.1%
Annual volatility: 10.0%
Sharpe Ratio: 2.61


(0.28096582225529926, 0.10000000046970839, 2.609658210295209)

In [571]:
#Target_return=0.6

ef2 = EfficientFrontier(mu, S, weight_bounds=(0, 1)) 
weights_2 = ef2.efficient_return(target_return=0.6, market_neutral=False)
ef2.portfolio_performance(verbose=True)

Expected annual return: 60.0%
Annual volatility: 19.6%
Sharpe Ratio: 2.97


(0.5999999999999983, 0.19560928303800937, 2.965094452533201)

In [572]:
PO2=ef2.clean_weights()

In [573]:
res2 = [d for d in PO2 if PO2[d] > 0]
res2

['DLR', 'EA', 'EW', 'HPE', 'MKTX', 'NVDA', 'STZ', 'TSN']

In [574]:
for i in res2:
    print(PO2.get(i)*100)

4.777
23.666999999999998
28.851
0.823
1.3299999999999998
33.127
2.465
4.96


In [576]:
#Minimum volatility portfolio


ef9 = EfficientFrontier(mu, S, weight_bounds=(0, 1)) 
weights= ef9.min_volatility()
ef9.portfolio_performance(verbose=True)

Expected annual return: 14.7%
Annual volatility: 9.2%
Sharpe Ratio: 1.39


(0.1473739306607184, 0.09189593094666287, 1.3860671451780304)

In [577]:
#Target volatilty =0.1+ ESG restriction for portfolio = 50

portfolio_min_score = 50

ef3 = EfficientFrontier(mu, S)
ef3.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef3.efficient_risk(target_volatility=0.1)
ef3.portfolio_performance()


(0.28096556942923934, 0.10000000049211542, 2.6096556814498753)

In [578]:
#Target volatilty =0.1+ ESG restriction for portfolio = 65

portfolio_min_score = 65

ef4 = EfficientFrontier(mu, S)
ef4.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef4.efficient_risk(target_volatility=0.1)
ef4.portfolio_performance()


(0.27150925557681854, 0.10000000009649, 2.5150925533413724)

In [579]:
portfolio_min_score = 80

ef5 = EfficientFrontier(mu, S)
ef5.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef5.efficient_risk(target_volatility=0.1)
ef5.portfolio_performance()

(0.21094867972844975, 0.10000000013120364, 1.9094867947791814)

# Regarding target volatility=0.1. When minimum weighted ESG score for portfolio = 0.5 the portfolio is the same. However when the minimum weighted ESG score requirement increased the expected annual return decreases and sharp ratio also decreases. 


In [580]:
# Target return = 0.6 + ESG constraints

In [581]:
# Target return = 0.6 + Env constraints >=50.

portfolio_min_score = 50

ef6 = EfficientFrontier(mu, S)
ef6.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef6.efficient_return(target_return=0.6, market_neutral=False)
ef6.portfolio_performance()

(0.5999999999999996, 0.19560928303801037, 2.9650944525331924)

In [582]:
# Target return = 0.6 + Env constraints >=60.

portfolio_min_score = 60

ef7 = EfficientFrontier(mu, S)
ef7.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef7.efficient_return(target_return=0.6, market_neutral=False)
ef7.portfolio_performance()

(0.5999568152335403, 0.19819431686246278, 2.9262030537232917)

In [584]:
# Target return = 0.6 + Env constraints >=67.

portfolio_min_score =67

ef8 = EfficientFrontier(mu, S)
ef8.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef8.efficient_return(target_return=0.6, market_neutral=False)
ef8.portfolio_performance()

(0.6000000000000004, 0.23320514081866883, 2.487080679113269)

# When setting target return 0.6 and E_concstraint equal to 50 the portfoilo is same as without constraints, but if we encrease E_constraint for portfolio sharp ratio drops and volatility increases.

In [585]:
#Min_volatility + EDG 90

portfolio_min_score =90

ef10 = EfficientFrontier(mu, S)
ef10.add_constraint(lambda w: list(merged_data['E_score'][1:375]) @ w >= portfolio_min_score)
weights = ef10.min_volatility()
ef10.portfolio_performance()

(0.14920661554667566, 0.10678911465528304, 1.2099230896684243)

# Minimum expected volatilty for portfolio with ESG constraint equal 90 is higher than for one without

# Conclusions

# 1) The main problem for ESG portfolio research is the data. 
# Even Bloomberg has very few reliable sources for ESG ratings. Most of the ratings have only latest value available for any given stock and lack of long term historical data.

# Even having acess to Bloomberg terminal you need to pay to have access  for some rich historical data. I assume a lot of companies can speculate and try to earn money providing their own ESG scores.

# Often ESG scores are very hard to interpret. 

# Different companies use different scales and different ranking are barely correlated

# We dont have a good reliable and undrstandable ESG score at this moment

# 2) Based on numerical results, if to set up minimum Robeco Total Sustainability Rank Envoronmental score to 50 the portfolio remains unchanged both in case of minimal variance, target volatility=0.1 and target risk return=0.6. That means that our portfolio without any environmental constraint is already at least "Half environmentally friendly" (50 is half of 100).

# When we rise environmental score limit:
# a)Target volatility 0.1: Sharpe ratio drops, expected return drops
# b)Expected return=0.6: Sharpe ratio drops, expected volatility increases
# c) Minimum volatility: minimum volatility for the portfoio with envirinmental restriction is higher comparing to unrestricted portfolio